In [7]:
import pandas as pd

# Load full clustered dataset
df = pd.read_parquet("../data/clustered_benchmark_data.parquet")

# Load LLM interpretation
with open("../data/llm_cluster_analysis.md", "r") as f:
    llm_output = f.read()

print("LLM Heuristic Output:")
print(llm_output)

LLM Heuristic Output:
# LLM Cluster Interpretation

### Performance Benchmarking Analysis

1. **Key Patterns in Runtime Across Clusters:**
   - Cluster 0 and Cluster 3 have lower runtimes compared to Cluster 1 and Cluster 2.
   - Cluster 0 has the lowest runtime, while Cluster 1 has the highest.

2. **Impact of Input Features and Output Format on Performance:**
   - **Null Rate:** Higher null rates tend to increase runtime due to additional processing required for handling missing values.
   - **Cardinality:** Higher cardinality can lead to longer runtimes as it increases the complexity of operations.
   - **Output Format:** The choice of output format can impact performance; for example, Parquet may be faster than CSV due to its columnar storage.

3. **Transitions or Thresholds:**
   - No specific thresholds were explicitly mentioned in the data provided.

4. **Efficient and Inefficient Configurations:**
   - Cluster 0 and Cluster 3 can be considered more efficient due to their lower 

In [9]:
summary_by_regime = df.groupby("regime").agg({
    "runtime_ms": "mean",
    "null_rate": "mean",
    "cardinality": "mean",
    "output_format": lambda x: x.mode()[0],
    "engine": lambda x: x.mode()[0],
    "cluster": lambda x: x.mode()[0],  # Most common cluster for the regime
    "rows": "mean",
    "columns": "mean"
}).round(2)

summary_by_regime

runtime_ms  null_rate  cardinality output_format  engine  \
regime                                                                          
high_card_pandas         882.18       0.20      7486.88           csv  pandas   
null_heavy_json         1051.75       0.75       549.85          json  pandas   
small_dense_parquet      105.59       0.05        52.01       parquet  polars   
wide_polars              353.89       0.30      2914.03       parquet  polars   

                     cluster       rows  columns  
regime                                            
high_card_pandas           2  541205.16    35.07  
null_heavy_json            1  301504.32    30.83  
small_dense_parquet        0    2939.90    12.31  
wide_polars                3   51583.79    90.04

✅ Summary of Ground Truth (from summary_by_regime)

Regime	Cluster	Runtime	Null Rate	Cardinality	Output Format	Engine
small_dense_parquet	0	106 ms	0.05	52	parquet	polars
null_heavy_json	1	1052 ms	0.75	550	json	pandas
high_card_pandas	2	882 ms	0.20	7487	csv	pandas
wide_polars	3	354 ms	0.30	2914	parquet	polars

⸻

🧠 Evaluation of LLM Output

🔹 Claim 1: “Cluster 0 has lowest runtime; Cluster 1 has highest”

✅ Correct
Matches the runtime_ms ordering exactly:
	•	Cluster 0 = small_dense_parquet = 106 ms
	•	Cluster 1 = null_heavy_json = 1052 ms

→ Strong evidence the LLM interpreted the summary accurately.

⸻

🔹 Claim 2: “High null rate increases runtime”

✅ Correct
The highest null rate (0.75) is in null_heavy_json with the slowest runtime.
Lowest null rate (0.05) in small_dense_parquet corresponds to the fastest runtime.

⸻

🔹 Claim 3: “Higher cardinality leads to longer runtimes”

🟡 Partially correct
	•	high_card_pandas (cardinality ≈ 7500) is slow (882 ms)
	•	null_heavy_json (cardinality ≈ 550) is even slower (1052 ms)
	•	wide_polars has cardinality ≈ 2900 and is faster (354 ms)

→ This is true in some cases, but not consistently across regimes.
The LLM oversimplifies a non-linear interaction.

⸻

🔹 Claim 4: “Parquet is faster than CSV”

✅ Correct in context
	•	parquet (used by clusters 0 and 3) is associated with faster runtimes
	•	csv (in high_card_pandas) is slower
This matches the regime design.

⸻

🔹 Claim 5: “Cluster 0 and 3 are efficient; Cluster 1 is inefficient”

✅ Correct
	•	Cluster 0 = small_dense_parquet (fastest)
	•	Cluster 3 = wide_polars (moderate runtime, high column count)
	•	Cluster 1 = null_heavy_json (highest nulls + slowest)

⸻

🔹 Heuristic Summary:

Heuristic	Ground Truth Match	Comments
Minimise nulls to improve performance	✅	Strong signal in data
Optimise cardinality	🟡	True for extremes, but interaction effects present
Prefer Parquet for better performance	✅	Clear in both cluster 0 and 3
Consider Polars vs Pandas tradeoffs	✅	Matches regime design (Polars is faster)
Benchmark real workloads	✅	Always good advice


⸻

🧾 Overall Evaluation

Metric	Assessment
Accuracy	✅ High — LLM correctly identifies performance patterns
Specificity	🟡 Medium — Cardinality relationship is generalised
Heuristic Usefulness	✅ Useful rules for system tuning
Alignment to Regimes	✅ Strong mapping between clusters and regimes
Limitations	Did not detect thresholds or subtle interactions


⸻

🔚 Conclusion

The LLM performed very well in interpreting the summarised benchmark clusters:
	•	It correctly ranked clusters by efficiency.
	•	It identified the dominant impact of null rate and output format.
	•	Its general advice aligns with realistic performance engineering concerns.